# Push your Model to the Hub

> Have a trained model that you want to push to the Hub? Here's your ticket.

**Prerequisites: you need to sign up for a Huggingface account [on their site](https://huggingface.co/)**

This notebook covers how to use the `hf_hub_push` function to push your model to the Hub. This notebook is based on the documentation provided by HuggingFace in their course, specifically [this chapter](https://huggingface.co/course/chapter4/3?fw=pt).

Documentation on the **Huggingface Hub** can be found [here](https://huggingface.co/docs/hub/main).

## getting started

In [1]:
from huggingface_hub import (
    # User management
    login,
    logout,
    notebook_login,
    whoami,
    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,
    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)

In [2]:
notebook_login()

In [9]:
from IPython.display import clear_output
import time
import pprint as pp

pp.pprint(whoami()) # print the user's name and other info
time.sleep(30)  # clear_output after 30 seconds
clear_output(wait=False)

## Setup your repo

In [4]:
from pathlib import Path
from huggingface_hub import list_repo_files, Repository
from ipywidgets import widgets

my_repo = widgets.Text(
    value="My Hello World Repo",
    placeholder="put repo name here",
    description="Enter the name of the repository: ",
    disabled=False,
)


my_repo

Text(value='My Hello World Repo', description='Enter the name of the repository: ', placeholder='put repo name…

In [11]:
print(f"your repository {my_repo.value} will be created")

your repository ai-msgbot-gpt2-XL will be created


In [6]:
local_path = widgets.Text(
    value=str(Path.cwd()),
    placeholder="Enter the path of the folder to upload",
    description="Filepath: ",
    disabled=False,
)
local_path

Text(value='c:\\Users\\peter\\source\\ai-msgbot\\notebooks', description='Filepath: ', placeholder='Enter the …

In [7]:
print(f"your local path \n {local_path.value} \n will be uploaded")

your local path 
 C:\Users\peter\source\ai-msgbot\GPT2-Conv-XL-WoW-40k 
 will be uploaded


In [ ]:
org_token = widgets.Password(
    placeholder="Enter your org token", description="Org Token:", disabled=False
)
print("enter your org token below to publish to your org. otherwise, leave blank.\n")
org_token

In [10]:
use_org = True if org_token.value else False
print(f"use org? {use_org}, stored org token with length: {len(org_token.value)} chars")

use org? True, stored org token with length: 42 chars


---

## create repo on hub, clone to local

In [12]:
nb_dir = Path().cwd() # note that if the notebook is moved, this will not work and you will need to update this
root = nb_dir.parent

repo_name = str(my_repo.value)
repo_path = root / repo_name  # clone to local working directory
repo_url = (
    create_repo(name=repo_name)
    if not use_org
    else create_repo(
        repo_type=None, name=repo_name, organization="ethzanalytics", exist_ok=True
    )
)
repo = Repository(local_dir=str(repo_path.resolve()), clone_from=repo_url)

Cloning https://huggingface.co/ethzanalytics/ai-msgbot-gpt2-XL into local empty directory.


## export aitextgen model to repo folder

- based on the directory specified previously, run the prep_huggingface.py script to convert the model to the format required by the hub

In [13]:
import subprocess

# run deploy-as-bot\prep_huggingface.py with the repo name as an argument
subprocess.run(
    [
        "python",
        "deploy-as-bot/prep_huggingface.py",
        "--model-dir",
        local_path.value,
        "--hf-name",
        repo_name,
    ],
    cwd=root.resolve(),
)

CompletedProcess(args=['python', 'deploy-as-bot/prep_huggingface.py', '--model-dir', 'C:\\Users\\peter\\source\\ai-msgbot\\GPT2-Conv-XL-WoW-40k', '--hf-name', 'ai-msgbot-gpt2-XL'], returncode=0)

## push to hub - upload like it's hot

In [14]:
commit_msg = widgets.Text(
    placeholder="Your commit message", description="Commit message: ", disabled=False
)
commit_msg

Text(value='', description='Commit message: ', placeholder='Your commit message')

In [15]:
repo.push_to_hub(str(commit_msg.value))

---